In [1]:
pip install going_modular

ERROR: Could not find a version that satisfies the requirement going_modular (from versions: none)
ERROR: No matching distribution found for going_modular
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [3]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torchinfo import summary
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT

In [7]:
pretrained_vit = torchvision.models.vit_b_16(weights = pretrained_vit_weights).to(device)

In [8]:
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False

In [9]:
pretrained_vit.heads = nn.Linear(in_features = 768, out_features = 7).to(device)

In [10]:
summary(model = pretrained_vit,
       input_size = (32, 3, 224, 224),
       col_names = ['input_size', 'output_size', 'num_params', 'trainable'],
       col_width = 20,
       row_settings = ['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
VisionTransformer (VisionTransformer)                        [32, 3, 224, 224]    [32, 7]              768                  Partial
├─Conv2d (conv_proj)                                         [32, 3, 224, 224]    [32, 768, 14, 14]    (590,592)            False
├─Encoder (encoder)                                          [32, 197, 768]       [32, 197, 768]       151,296              False
│    └─Dropout (dropout)                                     [32, 197, 768]       [32, 197, 768]       --                   --
│    └─Sequential (layers)                                   [32, 197, 768]       [32, 197, 768]       --                   False
│    │    └─EncoderBlock (encoder_layer_0)                   [32, 197, 768]       [32, 197, 768]       (7,087,872)          False
│    │    └─EncoderBlock (encoder_layer_1)                   [32, 197, 768]       [32, 

In [11]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("junkal/flowerdatasets")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/flowerdatasets


In [12]:
os.listdir('/kaggle/input/flowerdatasets')

['flowers']

In [13]:
os.listdir('/kaggle/input/flowerdatasets/flowers')

['val', 'test', 'train']

In [14]:
train_dir = '/kaggle/input/flowerdatasets/flowers/train'
test_dir = '/kaggle/input/flowerdatasets/flowers/test'
val_dir = '/kaggle/input/flowerdatasets/flowers/val'

In [15]:
pretrained_vit_transforms = pretrained_vit_weights.transforms()
print(pretrained_vit_transforms)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [16]:
NUM_WORKERS = os.cpu_count()

In [17]:
def create_dataloaders(
    train_dir: str,
    val_dir: str,
    transform: transforms.Compose,
    batch_size = int,
    num_workers: int=NUM_WORKERS
):
    train_data = datasets.ImageFolder(train_dir, transform = transform)
    val_data = datasets.ImageFolder(val_dir, transform = transform)
    class_names = train_data.classes
    train_dataloader = DataLoader(
        train_data,
        batch_size = batch_size,
        shuffle = True,
        num_workers = num_workers,
        pin_memory = True
    )
    val_dataloader = DataLoader(
        val_data,
        batch_size = batch_size,
        shuffle = True,
        num_workers = num_workers,
        pin_memory = True
    )

    return train_dataloader, val_dataloader, class_names

In [18]:
train_dataloader_pretrained, val_dataloader_pretrained, class_names = create_dataloaders(train_dir = train_dir,
                                                                                        val_dir = val_dir,
                                                                                        transform = pretrained_vit_transforms,
                                                                                        batch_size = 32)

In [19]:
optimizer = torch.optim.Adam(params = pretrained_vit.parameters(), lr = 1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

In [20]:
pretrained_vit_results = engine.train(model = pretrained_vit,
                                     train_dataloader = train_dataloader_pretrained,
                                     test_dataloader = val_dataloader_pretrained,
                                     optimizer = optimizer,
                                     loss_fn = loss_fn,
                                     epochs = 10,
                                     device = device)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.2126 | train_acc: 0.9579 | test_loss: 0.0617 | test_acc: 0.9906
Epoch: 2 | train_loss: 0.0463 | train_acc: 0.9916 | test_loss: 0.0390 | test_acc: 0.9932
Epoch: 3 | train_loss: 0.0286 | train_acc: 0.9953 | test_loss: 0.0279 | test_acc: 0.9953
Epoch: 4 | train_loss: 0.0199 | train_acc: 0.9973 | test_loss: 0.0240 | test_acc: 0.9961
Epoch: 5 | train_loss: 0.0143 | train_acc: 0.9983 | test_loss: 0.0202 | test_acc: 0.9953
Epoch: 6 | train_loss: 0.0112 | train_acc: 0.9985 | test_loss: 0.0187 | test_acc: 0.9953
Epoch: 7 | train_loss: 0.0086 | train_acc: 0.9994 | test_loss: 0.0157 | test_acc: 0.9966
Epoch: 8 | train_loss: 0.0069 | train_acc: 0.9996 | test_loss: 0.0146 | test_acc: 0.9966
Epoch: 9 | train_loss: 0.0057 | train_acc: 0.9999 | test_loss: 0.0140 | test_acc: 0.9966
Epoch: 10 | train_loss: 0.0046 | train_acc: 1.0000 | test_loss: 0.0134 | test_acc: 0.9966
